In [2]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer
import torch
from utils import util
import json

# Load models
checkpoint_path = "/Users/alvinchen/Documents/GitHub/brainteaser-data/testing/models/bert-BASED-uncased/"
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForMultipleChoice.from_pretrained(checkpoint_path)

In [3]:
# Load datasets
with open("../data/new_test_data_nolabel/SP_new_test_corrected.json") as f:
    data = f.read()
data = json.loads(data)

In [4]:
# Evaluate dev set
def predict(model, tokenizer, dataset):
    preds = []
    model.eval()
    for sample in dataset:
        inputs = [[sample["question"], answer] for answer in sample["choice_list"]]
        tokens = tokenizer(inputs, return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0)
        outputs = model(**{k: v.unsqueeze(0) for k, v in tokens.items()}, labels=labels)
        logits = outputs.logits
        pred = logits.argmax().item()
        preds.append(pred)
    return preds

preds = predict(model, tokenizer, data)

In [9]:
with open("../data/new_test_data_nolabel/sp_choices.txt", "r") as f:
    valid_labs = f.readlines()
for i in range(len(valid_labs)):
    valid_labs[i] = int(valid_labs[i].strip())

In [10]:
f1 = util.avg_f1_score(preds, valid_labs, [0,1,2,3])
accuracy = util.accuracy(preds, valid_labs)
print("Accuracy: ", accuracy)
print("F1: ", f1)

Accuracy:  0.7416666666666667
F1:  0.7211590659834428


120
